In [2]:
import os
import requests
import json
from typing import List
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
# Initialize and constants
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'API') #PUT UR API HERE
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [4]:

class Website:

    url: str
    title: str
    body: str
    links: List[str]
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
ed = Website("https://www.twitch.tv")
print(ed.get_contents())

Webpage Title:
Twitch
Webpage Contents:





In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
def get_links(url):
    website = Website(url)
    completion = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = completion.choices[0].message.content
    return json.loads(result)

In [10]:
get_links("https://www.twitch.tv")

{'links': [{'type': 'about page', 'url': 'https://www.twitch.tv/p/about'},
  {'type': 'careers page', 'url': 'https://www.twitch.tv/jobs'}]}

In [11]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [12]:
print(get_all_details("https://www.twitch.tv"))

Found links: {'links': [{'type': 'about page', 'url': 'https://www.twitch.tv/p/about'}, {'type': 'careers page', 'url': 'https://www.twitch.tv/p/careers'}]}
Landing page:
Webpage Title:
Twitch
Webpage Contents:




about page
Webpage Title:
Redirecting...
Webpage Contents:
Redirecting…
Click here if you are not redirected.



careers page
Webpage Title:
404 Not Found
Webpage Contents:
404 Not Found
Code: NoSuchKey
Message: The specified key does not exist.
Key: p/careers
RequestId: 9Q8JZZJKJQQXPF2Y
HostId: 5o5sJUdj4Uxtksb9SOkxRvoBUCUunbC5jFfdzD2g9SoUcehrEb/dFToAMJnrPwyuKxMn+YEimQg=
An Error Occurred While Attempting to Retrieve a Custom Error Document
Code: NoSuchKey
Message: The specified key does not exist.
Key: index.html




In [13]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [14]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000]
    return user_prompt

In [15]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [16]:
create_brochure("Twitch", "https://www.twitch.tv")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.twitch.tv/p/about'}, {'type': 'careers page', 'url': 'https://www.twitch.tv/jobs'}, {'type': 'company page', 'url': 'https://www.twitch.tv/p/about#company'}]}


# Twitch Brochure

---

## Welcome to Twitch

Twitch is the leading platform for live community engagement, connecting millions of creators and viewers around the world. Through innovative technology and passionate community support, we empower individuals to share their passions and connect through live entertainment.

## Our Company Culture

At Twitch, we believe in the power of community, creativity, and inclusivity. Our team is built on the foundation of collaboration, diversity, and mutual respect. Every member of our team plays a vital role, and we are committed to creating an environment where everyone’s voice is heard.

### Key Cultural Elements:
- **Inclusivity**: We actively work to create an equitable workplace where everyone, regardless of background, feels welcome and valued.
- **Innovation**: We embrace change and continuously strive for improvement, encouraging creative solutions to challenges.
- **Community Focus**: We not only serve communities but are part of them, understanding the importance of real connections.

## Our Customers

Twitch caters to a vibrant global community of streamers and viewers, including gamers, musicians, artists, and creators from diverse fields. Our platform serves as a launchpad for creators to showcase their talent, engage with fans, and monetize their content through subscriptions, donations, and brand partnerships.

### Notable Features:
- **DJ Program**: A first-of-its-kind initiative allowing DJs to stream popular music safely and legally, helping them build and engage their communities.
- **Mod Appreciation Day**: Celebrating the important role of moderators who help cultivate safe and welcoming environments for viewers.

## Careers at Twitch

If you're passionate about creating connections and building tools for communities, Twitch is the place for you! We're always on the lookout for innovative thinkers to join our diverse team.

### Opportunities for Growth:
- **Early Career Programs**: Internships, apprenticeships, and programs for new graduates to kickstart your career in the tech and entertainment industries.
- **Professional Development**: We offer workshops, mentorship programs, and a supportive environment for continuous learning.
- **Flexible Work Arrangements**: Enjoy the benefits of remote work with a focus on wellness and work-life balance.

### Benefits and Perks:
- Competitive health and wellness programs
- Paid parental leave
- Opportunities for mentorship and career growth

## Join Us!

We are on a mission to empower creators and build the future of live entertainment. With a commitment to community and inclusivity, Twitch is the place where you can make a real impact.

**Visit us at [twitch.tv/careers](https://twitch.tv/careers) to learn more about job openings and our culture.**

---

**Follow Us:**
- [Twitter](https://twitter.com/twitch)
- [Facebook](https://facebook.com/twitch)
- [Instagram](https://instagram.com/twitch)

We look forward to welcoming you to the Twitch community!

In [17]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [18]:
stream_brochure("Twitch", "https://www.twitch.tv")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.twitch.tv/p/about'}, {'type': 'careers page', 'url': 'https://www.twitch.tv/jobs'}]}



# Welcome to Twitch

## About Us
At **Twitch**, we're on a mission to empower and connect live communities on the internet. Whether through gaming, music, or creative content, we strive to create the future of live entertainment. Our platform serves millions of users worldwide, providing them with a space to share their passions and build vibrant communities. 

## Our Culture
Twitch thrives on **diversity, collaboration, and inclusion**. We believe that our strength lies in our people, and we work hard to create a workplace where everyone feels valued and heard. Our initiatives around **Equity & Inclusion** highlight our commitment to fostering a welcoming environment for all employees. From employee-led Guilds to programs that encourage mentorship and community connection, we prioritize a supportive culture.

### Employee Story
Zoe Willis from the Twitch Music Operations team shares, "Being part of the Twitch family has empowered me to help artists build their communities while connecting with my colleagues through engaging events and initiatives." 

## Customers
Our platform caters to a wide range of **communities**, from gamers to musicians and beyond. We create opportunities for creators to thrive, such as our innovative **DJ Program**, which facilitates safe and legal music streaming for DJs on Twitch. We also celebrate our moderating community's essential role, ensuring a welcoming atmosphere for all.

## Careers at Twitch
Join our quest! At Twitch, we offer exciting career opportunities across various domains, including technology, marketing, and more. Our **Early Career Program** is designed for interns and recent grads to kickstart their careers in a dynamic environment. 

### Why Work for Twitch?
- **Wellness Benefits**: We prioritize our employees' mental and physical wellness with robust health programs, flexible work policies, and paid parental leave.
- **Development Opportunities**: We invest in our team with coaching, mentorship programs, and continuous learning opportunities.
- **Remote-Friendly Work**: With global no-meeting days and modified work arrangements, you can create a work schedule that fits your needs.

## Join Us
Explore our current job openings and become a part of our vibrant community-building journey. Whether you're a seasoned professional or just starting out, there's a place for you at Twitch.

**Visit us** at [Twitch Careers](https://www.twitch.tv) to learn more.

---
*Together, let's shape the future of live communities!*


In [19]:
stream_brochure("Twitch", "https://www.twitch.tv")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.twitch.tv/p/about'}, {'type': 'careers page', 'url': 'https://www.twitch.tv/p/jobs'}, {'type': 'company page', 'url': 'https://www.twitch.tv/company'}]}


# Twitch Company Brochure

## Welcome to Twitch

Twitch is a leading interactive live streaming service, primarily focused on gaming, but expanding into music, creative arts, and sports. With millions of users interacting and engaging daily, Twitch has become a cornerstone of online entertainment and community building.

---

### About Twitch

Twitch's mission is to empower creators and their communities. Our platform provides broadcasters with the tools they need to connect with their audience in real-time, fostering a unique environment that combines gaming, entertainment, and social interaction.

---

### Company Culture

At Twitch, we pride ourselves on our inclusive and diverse culture. We value creativity, collaboration, and innovation. Our team members are encouraged to pursue their passions, whether they be in gaming, technology, or other areas of interest. We believe in working hard and playing hard, which is reflected in our dynamic workplace and various employee initiatives.

- **Inclusivity**: We celebrate diversity and strive to create an environment where everyone feels welcome.
- **Collaboration**: Teamwork is at the heart of everything we do.
- **Creativity**: We provide the freedom to explore new ideas and push boundaries.

---

### Our Customers

Twitch serves a vast array of users, including:

- **Gamers**: From casual players to professional esports athletes, Twitch offers a space to share gameplay and connect with others.
- **Content Creators**: Musicians, artists, and other creatives can showcase their talent and build a community around their work.
- **Viewers**: Anyone looking for live entertainment and the thrill of interactive experiences can find a niche on Twitch.

---

### Careers at Twitch

Twitch is always looking for ambitious and talented individuals to join our innovative team. Explore career opportunities in various departments, including:

- **Engineering**
- **Content Creation**
- **Marketing and Sales**
- **Community Management**
- **Design and User Experience**

By joining Twitch, you'll have the chance to work in a fast-paced environment and make a real impact in the world of live streaming and online entertainment.

If you're passionate about gaming, technology, and community, we want to hear from you!

---

For more information about Twitch, including potential career opportunities, stay tuned to our website, where we'll provide forthcoming updates.

*Join us in transforming the landscape of online entertainment.*